# StyleGAN

The purpose of this notebook is to better understand StyleGAN and enable generation of video by interpelating within a latent space.

The most recent version of this notebook was developed by Josh Urban Davis (https://joshurbandavis.com/, @joshurbandavis). See my GitHub repository for pre-trained models or my fine-tuning notebook for methods to modify pre-trained models.

In [ ]:
!git clone https://github.com/NVlabs/stylegan.git
%cd stylegan

Cloning into 'stylegan'...
remote: Enumerating objects: 77, done.
remote: Total 77 (delta 0), reused 0 (delta 0), pack-reused 77
Unpacking objects: 100% (77/77), done.
/content/stylegan


In [ ]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import config

tflib.init_tf()

# Load pre-trained network.
url = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ'
with dnnlib.util.open_url(url, cache_dir=config.cache_dir) as f:
    _G, _D, Gs = pickle.load(f)
    # _G = Instantaneous snapshot of the generator. Mainly useful for resuming a previous training run.
    # _D = Instantaneous snapshot of the discriminator. Mainly useful for resuming a previous training run.
    # Gs = Long-term average of the generator. Yields higher-quality results than the instantaneous snapshot.


W0703 00:26:18.898688 140519557355392 deprecation_wrapper.py:119] From /content/stylegan/dnnlib/tflib/tfutil.py:34: The name tf.Dimension is deprecated. Please use tf.compat.v1.Dimension instead.

W0703 00:26:18.901242 140519557355392 deprecation_wrapper.py:119] From /content/stylegan/dnnlib/tflib/tfutil.py:74: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0703 00:26:18.902614 140519557355392 deprecation_wrapper.py:119] From /content/stylegan/dnnlib/tflib/tfutil.py:128: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0703 00:26:18.919707 140519557355392 deprecation_wrapper.py:119] From /content/stylegan/dnnlib/tflib/tfutil.py:97: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0703 00:26:18.920955 140519557355392 deprecation_wrapper.py:119] From /content/stylegan/dnnlib/tflib/tfutil.py:109: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.s

W0703 00:26:30.908653 140519557355392 deprecation.py:323] From <string>:364: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


**The section below allows for downloading of model data**

# This section used for generating images and downloading them locally

In [ ]:
mkdir generated_faces_test

In [ ]:
# Pick latent vector.
rnd = np.random.RandomState(100)
latents = rnd.randn(1, Gs.input_shape[1])

# Generate image.
fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
images = Gs.run(latents, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)

In [ ]:
import scipy.misc


root_dir = '/content/stylegan/generated_faces_test/'


for num_pics in range(1000,1100):
  # Pick latent vector.
  rnd = np.random.RandomState(num_pics)
  latents = rnd.randn(1, Gs.input_shape[1])

  fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
  images = Gs.run(latents, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)

  for i, images in enumerate(images):
    filename = '%sframe_%05d.png'%(root_dir, num_pics+1)
    scipy.misc.imsave(filename, images)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  app.launch_new_instance()


In [ ]:
from google.colab import files
!zip -r generated_faces_1.zip generated_faces



In [ ]:
files.download('generated_faces_1.zip')

# This section generates images and saves them directly to gogle drive.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import scipy.misc

root_dir = '/content/gdrive/My Drive/generatedFaces/'

for num_pics in range(9025,15000):
  # Pick latent vector.
  rnd = np.random.RandomState(num_pics)
  latents = rnd.randn(1, Gs.input_shape[1])

  fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
  images = Gs.run(latents, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)

  for i, images in enumerate(images):
    filename = '%sframe_%05d.png'%(root_dir, num_pics+1)
    scipy.misc.imsave(filename, images)

AttributeError: ignored

# **Below is mostly a reference for the above code**

In [ ]:
import scipy.misc

root_dir = '/content/stylegan/generated_faces/'
for i, images in enumerate(images):
    filename = '%sframe_%05d.png'%(root_dir, i+1)
    scipy.misc.imsave(filename, images)

# Random interpolation grid

Based on the ProGAN interpolation code
https://github.com/tkarras/progressive_growing_of_gans/blob/master/misc.py

In [ ]:
import scipy

grid_size = [2,2]
image_shrink = 1
image_zoom = 1
duration_sec = 10.0
smoothing_sec = 1.0
mp4_fps = 60
mp4_codec = 'libx264'
mp4_bitrate = '16M'
random_seed = 404
mp4_file = 'random_grid_%s.mp4' % random_seed
minibatch_size = 8

num_frames = int(np.rint(duration_sec * mp4_fps))
random_state = np.random.RandomState(random_seed)

# Generate latent vectors
shape = [num_frames, np.prod(grid_size)] + Gs.input_shape[1:] # [frame, image, channel, component]
all_latents = random_state.randn(*shape).astype(np.float32)
all_latents = scipy.ndimage.gaussian_filter(all_latents, [smoothing_sec * mp4_fps] + [0] * len(Gs.input_shape), mode='wrap')
all_latents /= np.sqrt(np.mean(np.square(all_latents)))


def create_image_grid(images, grid_size=None):
    assert images.ndim == 3 or images.ndim == 4
    num, img_h, img_w, channels = images.shape

    if grid_size is not None:
        grid_w, grid_h = tuple(grid_size)
    else:
        grid_w = max(int(np.ceil(np.sqrt(num))), 1)
        grid_h = max((num - 1) // grid_w + 1, 1)

    grid = np.zeros([grid_h * img_h, grid_w * img_w, channels], dtype=images.dtype)
    for idx in range(num):
        x = (idx % grid_w) * img_w
        y = (idx // grid_w) * img_h
        grid[y : y + img_h, x : x + img_w] = images[idx]
    return grid

# Frame generation func for moviepy.
def make_frame(t):
    frame_idx = int(np.clip(np.round(t * mp4_fps), 0, num_frames - 1))
    latents = all_latents[frame_idx]
    fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    images = Gs.run(latents, None, truncation_psi=0.7,
                          randomize_noise=False, output_transform=fmt)

    grid = create_image_grid(images, grid_size)
    if image_zoom > 1:
        grid = scipy.ndimage.zoom(grid, [image_zoom, image_zoom, 1], order=0)
    if grid.shape[2] == 1:
        grid = grid.repeat(3, 2) # grayscale => RGB
    return grid

# Generate video.
import moviepy.editor
video_clip = moviepy.editor.VideoClip(make_frame, duration=duration_sec)
video_clip.write_videofile(mp4_file, fps=mp4_fps, codec=mp4_codec, bitrate=mp4_bitrate)



# Style mixing

## Coarse

In [ ]:
import scipy

duration_sec = 30.0
smoothing_sec = 1.0
mp4_fps = 40

num_frames = int(np.rint(duration_sec * mp4_fps))
random_seed = 500
random_state = np.random.RandomState(random_seed)


w = 512
h = 512
#src_seeds = [601]
dst_seeds = [700]
style_ranges = ([0] * 7 + [range(8,16)]) * len(dst_seeds)

fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
synthesis_kwargs = dict(output_transform=fmt, truncation_psi=0.7, minibatch_size=8)

shape = [num_frames] + Gs.input_shape[1:] # [frame, image, channel, component]
src_latents = random_state.randn(*shape).astype(np.float32)
src_latents = scipy.ndimage.gaussian_filter(src_latents,
                                            smoothing_sec * mp4_fps,
                                            mode='wrap')
src_latents /= np.sqrt(np.mean(np.square(src_latents)))

dst_latents = np.stack(np.random.RandomState(seed).randn(Gs.input_shape[1]) for seed in dst_seeds)


src_dlatents = Gs.components.mapping.run(src_latents, None) # [seed, layer, component]
dst_dlatents = Gs.components.mapping.run(dst_latents, None) # [seed, layer, component]
src_images = Gs.components.synthesis.run(src_dlatents, randomize_noise=False, **synthesis_kwargs)
dst_images = Gs.components.synthesis.run(dst_dlatents, randomize_noise=False, **synthesis_kwargs)


canvas = PIL.Image.new('RGB', (w * (len(dst_seeds) + 1), h * 2), 'white')

for col, dst_image in enumerate(list(dst_images)):
    canvas.paste(PIL.Image.fromarray(dst_image, 'RGB'), ((col + 1) * h, 0))

def make_frame(t):
    frame_idx = int(np.clip(np.round(t * mp4_fps), 0, num_frames - 1))
    src_image = src_images[frame_idx]
    canvas.paste(PIL.Image.fromarray(src_image, 'RGB'), (0, h))

    for col, dst_image in enumerate(list(dst_images)):
        col_dlatents = np.stack([dst_dlatents[col]])
        col_dlatents[:, style_ranges[col]] = src_dlatents[frame_idx, style_ranges[col]]
        col_images = Gs.components.synthesis.run(col_dlatents, randomize_noise=False, **synthesis_kwargs)
        for row, image in enumerate(list(col_images)):
            canvas.paste(PIL.Image.fromarray(image, 'RGB'), ((col + 1) * h, (row + 1) * w))
    return np.array(canvas)

# Generate video.
import moviepy.editor
mp4_file = '/tmp/output.mp4'
mp4_codec = 'libx264'
mp4_bitrate = '16M'

video_clip = moviepy.editor.VideoClip(make_frame, duration=duration_sec)
video_clip.write_videofile(mp4_file, fps=mp4_fps, codec=mp4_codec, bitrate=mp4_bitrate)

## Fine

In [ ]:
import scipy

duration_sec = 20.0
smoothing_sec = 1.0
mp4_fps = 30

num_frames = int(np.rint(duration_sec * mp4_fps))
random_seed = 503
random_state = np.random.RandomState(random_seed)


w = 512
h = 512
style_ranges = [range(6,16)]

fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
synthesis_kwargs = dict(output_transform=fmt, truncation_psi=0.7, minibatch_size=8)

shape = [num_frames] + Gs.input_shape[1:] # [frame, image, channel, component]
src_latents = random_state.randn(*shape).astype(np.float32)
src_latents = scipy.ndimage.gaussian_filter(src_latents,
                                            smoothing_sec * mp4_fps,
                                            mode='wrap')
src_latents /= np.sqrt(np.mean(np.square(src_latents)))

dst_latents = np.stack([random_state.randn(Gs.input_shape[1])])


src_dlatents = Gs.components.mapping.run(src_latents, None) # [seed, layer, component]
dst_dlatents = Gs.components.mapping.run(dst_latents, None) # [seed, layer, component]


def make_frame(t):
    frame_idx = int(np.clip(np.round(t * mp4_fps), 0, num_frames - 1))
    col_dlatents = np.stack([dst_dlatents[0]])
    col_dlatents[:, style_ranges[0]] = src_dlatents[frame_idx, style_ranges[0]]
    col_images = Gs.components.synthesis.run(col_dlatents, randomize_noise=False, **synthesis_kwargs)
    return col_images[0]

# Generate video.
import moviepy.editor
mp4_file = 'fine_%s.mp4' % (random_seed)
mp4_codec = 'libx264'
mp4_bitrate = '16M'

video_clip = moviepy.editor.VideoClip(make_frame, duration=duration_sec)
video_clip.write_videofile(mp4_file, fps=mp4_fps, codec=mp4_codec, bitrate=mp4_bitrate)